In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from utils import load_position_dataframe
full_df = pd.read_csv('../LinearMobility_results.csv')

def build_position_dataframe(model_, seed_, angle_, speed_, nodes_, broadcast_times, log_position_time, window_threshold_time, sim_time):
    path = f'../{model_}/seed{seed_}_time{time}_angle{angle_}_speed{speed_}'
    common_columns = ['timestamp', 'frequency', 'sim_time', 'broadcast_time', 'log_position_time']
    this_df = pd.DataFrame()
    for bt in broadcast_times:
        partial_df = load_position_dataframe(nodes_[0], bt, log_position_time, window_threshold_time, sim_time, path)
        for node in nodes_:
            if node!=nodes_[0]:
                df = load_position_dataframe(node, bt, log_position_time, window_threshold_time, sim_time, path)
                partial_df = partial_df.merge(df, how='outer', left_on=common_columns, right_on=common_columns).fillna(0)
            # partial_df = pd.concat([partial_df, df], axis=1)
        partial_df['seed'] = seed_
        partial_df['time'] = sim_time
        partial_df['angle'] = angle_
        partial_df['speed'] = speed_
        this_df = pd.concat([this_df, partial_df])
    return this_df

In [2]:
time = 60
seed = 20
speeds = [0.5, 1, 2, 4]
angle = 30
model = 'LinearMobility'
bts = list(np.arange(1, 20.5, 0.5)) #seconds
lp = 0.1 #seconds
wt = 5 #seconds
nodes = [1, 2]

full_position_df = pd.DataFrame()
for speed in speeds:
    position_df = build_position_dataframe(model, seed, angle, speed, nodes, bts, lp, wt, time)
    full_position_df = pd.concat([full_position_df, position_df])


AttributeError: 'DataFrame' object has no attribute 'x_pos_A'

In [15]:
full_position_df['distance'] = (np.sqrt(np.power(full_position_df.x_pos_1-full_position_df.x_pos_2, 2)+np.power(full_position_df.y_pos_1-full_position_df.y_pos_2, 2)))<2.0
adj_check = (full_position_df.distance != full_position_df.distance.shift()).cumsum()
time = full_position_df.groupby(['node_1','distance', 'speed', 'angle', 'broadcast_time', adj_check], as_index=False, sort=False)['distance'].sum()*0.1
real_contact_time = time.groupby(['node_1', 'speed', 'angle', 'broadcast_time']).sum()
real_contact_time

distance
node_1 speed angle broadcast_time          
0.1    0.05  3.0   0.10              1230.3
                   0.15              1230.3
                   0.20              1230.3
                   0.25              1230.3
                   0.30              1230.3
...                                     ...
       0.40  3.0   1.80              1237.5
                   1.85              1237.5
                   1.90              1237.5
                   1.95              1237.5
                   2.00              1237.5

[156 rows x 1 columns]

In [ ]:
time = 60
seed = 20
speed = 0.5
angles = [15, 30, 60, 90]

angles_df = full_df[(full_df['time']==time) & (full_df['seed']==seed) & (full_df['speed']==speed) & (full_df['angle'].isin(angles))]
angles_df = angles_df.drop(["told_by", "hop", "start", "end", "sim_time"], axis=1).groupby(["node", "seen_id", "broadcast_time", "frequency", "angle", "seed", "speed", "time"], as_index=False).sum()

a4_dims = (16,9)
fig, ax = plt.subplots(figsize=a4_dims)
plot = sns.lineplot(data=angles_df[['frequency', 'angle', 'window_duration']], x="frequency", y="window_duration", hue='angle')
plot.set(xscale='log')
plt.show()

In [ ]:
time = 60
seed = 20
speeds = [0.5, 1, 2, 4]
angle = 30

speeds_df = full_df[(full_df['time']==time) & (full_df['seed']==seed) & (full_df['speed'].isin(speeds)) & (full_df['angle']==angle)]
speeds_df = speeds_df.drop(["told_by", "hop", "start", "end", "sim_time"], axis=1).groupby(["node", "seen_id", "broadcast_time", "frequency", "angle", "seed", "speed", "time"], as_index=False).sum()

a4_dims = (16,9)
fig, ax = plt.subplots(figsize=a4_dims)
plot = sns.lineplot(data=speeds_df[['frequency', 'speed', 'window_duration']], x="frequency", y="window_duration", hue='speed')
plot.set(xscale='log')
plt.show()

In [ ]:
full_position_df